## Test Género
En este notebook, analizaremos si es posible mejorar los modelos clasificando las canciones por géneros.

In [1]:
import pandas as pd

In [2]:
df_final=pd.read_csv('df_final.csv')

In [3]:
df_final['Genero']

0              italian hip hop, italian pop, trap italiana
1                                                      NaN
2        ambient, braindance, electronica, intelligent ...
3                           jam band, neo mellow, pop rock
4                        argentine hip hop, trap argentino
                               ...                        
16861                                        j-pop, j-rock
16862                                                k-pop
16863                                    k-pop, korean r&b
16864                              k-pop, k-pop girl group
16865                                    k-pop, korean r&b
Name: Genero, Length: 16866, dtype: object

In [4]:
df_final['Stream'].describe()

count    1.686600e+04
mean     1.362465e+08
std      2.433885e+08
min      6.574000e+03
25%      1.748466e+07
50%      4.993142e+07
75%      1.391041e+08
max      3.386520e+09
Name: Stream, dtype: float64

Aquí vamos a ordenar en oreden descendente los géneros por numero de canciones

In [5]:
df_generos = df_final.assign(Genero=df_final['Genero'].str.split(', ')).explode('Genero')



In [6]:
# Agrupar por 'Genero' y contar el número de tracks
df_generos_count = df_generos.groupby('Genero', as_index=False).size()

# Renombrar las columnas para mayor claridad
df_generos_count.columns = ['Genero', 'Num_Tracks']

# Ordenar por número de tracks en orden descendente
df_generos_count = df_generos_count.sort_values(by='Num_Tracks', ascending=False)

# Reiniciar el índice para un formato limpio
df_generos_count.reset_index(drop=True, inplace=True)

# Mostrar el DataFrame
print(df_generos_count)



                           Genero  Num_Tracks
0                            rock        1565
1                             pop        1549
2                             rap        1184
3                       dance pop         887
4                         hip hop         816
...                           ...         ...
1107          neue deutsche harte           3
1108                uk doom metal           2
1109  australian children's music           2
1110                   afghan pop           2
1111                rock keyboard           1

[1112 rows x 2 columns]


In [7]:
df_generos_count.head(20)

,Genero,Num_Tracks
0,rock,1565
1,pop,1549
2,rap,1184
3,dance pop,887
4,hip hop,816
5,latin pop,777
6,classic rock,773
7,filmi,684
8,soft rock,683
9,album rock,650


In [8]:
df_final['Genero'] = df_final['Genero'].apply(
    lambda x: [genero.strip() for genero in x.split(',')]  # Elimina espacios
    if isinstance(x, str) 
    else []  # Maneja NaN u otros tipos (asignando lista vacía)
)

In [9]:
df_final['Genero']

0            [italian hip hop, italian pop, trap italiana]
1                                                       []
2        [ambient, braindance, electronica, intelligent...
3                         [jam band, neo mellow, pop rock]
4                      [argentine hip hop, trap argentino]
                               ...                        
16861                                      [j-pop, j-rock]
16862                                              [k-pop]
16863                                  [k-pop, korean r&b]
16864                            [k-pop, k-pop girl group]
16865                                  [k-pop, korean r&b]
Name: Genero, Length: 16866, dtype: object

Esta función me ofrece si hay un género específico en la lista de generos

In [10]:
def busca_genero(genero_buscado, lista_generos):
    # Si el valor es NaN (tipo float), retornar 0
    if isinstance(lista_generos, float):
        return 0
    # Si es una lista o cadena, verificar el género
    if genero_buscado in lista_generos:
        return 1
    else:
        return 0

Vamos a analizar los modelos más importantes para los 20 principales géneros utilizando LazyClassifier

In [12]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from lazypredict.Supervised import LazyClassifier

# Lista de generos a evaluar
generos = df_generos_count.head(20)['Genero']

# Diccionario para almacenar resultados
test = {}

for genero in generos:
    # Filtrar por genero y hacer una copia para evitar errores
    
    df_temp = df_final.copy()
    df_temp[genero] = df_temp['Genero'].apply(lambda x: busca_genero(genero, x))
    df_gen = df_temp[df_temp[genero] == 1].copy()
    
    
    

    # Calcular el tercer cuartil (Q3)
    q3 = df_gen['Stream'].quantile(0.75)

    # Definir las categorías
    bins = [0, q3, df_gen['Stream'].max()]
    labels = [0, 1]

    df_gen['Categoria'] = pd.cut(df_gen['Stream'], bins=bins, labels=labels, include_lowest=True)

    # Verificar si hay suficientes muestras en la categoría 1
    count_cat1 = df_gen['Categoria'].value_counts().get(1, 0)

    if count_cat1 == 0:
        print(f"⚠️ genero {genero}: No hay suficientes muestras en la categoría 1, saltando...")
        continue

    # Filtrar por categorías
    categoria_0 = df_gen[df_gen['Categoria'] == 0]
    categoria_1 = df_gen[df_gen['Categoria'] == 1]

    # Muestrear la categoría 0 con el mismo número de elementos que la categoría 1
    categoria_0_sample = categoria_0.sample(n=min(count_cat1, len(categoria_0)), random_state=42)

    # Concatenar para obtener un dataset balanceado
    df_balanceado = pd.concat([categoria_0_sample, categoria_1]).sample(frac=1, random_state=42).reset_index(drop=True)

    # Definir variables predictoras y objetivo
    X = df_balanceado[['Acousticness', 'Danceability', 'Duration_min', 'Energy',
                       'Instrumentalness', 'Key', 'Liveness', 'Loudness',
                       'Speechiness', 'Tempo', 'Valence']]
    y = df_balanceado['Categoria']

    # Dividir en entrenamiento y prueba
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Evaluación con LazyClassifier
    lazy_clf = LazyClassifier(verbose=0, ignore_warnings=False, custom_metric=None)
    models, predictions = lazy_clf.fit(X_train, X_test, y_train, y_test)

    # Guardar resultados en el diccionario
    test[genero] = models

print("✅ Evaluación completada para todos los generos.")

 28%|██▊       | 9/32 [00:00<00:00, 24.01it/s]

CategoricalNB model failed to execute
Negative values in data passed to CategoricalNB (input X).
FixedThresholdClassifier model failed to execute
FixedThresholdClassifier.__init__() missing 1 required positional argument: 'estimator'


 72%|███████▏  | 23/32 [00:00<00:00, 25.41it/s]

SelfTrainingClassifier model failed to execute
You must pass an estimator to SelfTrainingClassifier. Use `estimator`.
StackingClassifier model failed to execute
StackingClassifier.__init__() missing 1 required positional argument: 'estimators'
TunedThresholdClassifierCV model failed to execute
TunedThresholdClassifierCV.__init__() missing 1 required positional argument: 'estimator'


100%|██████████| 32/32 [00:01<00:00, 21.67it/s]


...

SelfTrainingClassifier model failed to execute
You must pass an estimator to SelfTrainingClassifier. Use `estimator`.
StackingClassifier model failed to execute
StackingClassifier.__init__() missing 1 required positional argument: 'estimators'
TunedThresholdClassifierCV model failed to execute
TunedThresholdClassifierCV.__init__() missing 1 required positional argument: 'estimator'
XGBClassifier model failed to execute
'super' object has no attribute '__sklearn_tags__'
[LightGBM] [Info] Number of positive: 105, number of negative: 99
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000045 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 646
[LightGBM] [Info] Number of data points in the train set: 204, number of used features: 11
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.514706 -> initscore=0.058841
[LightGBM] [Info] Start training from score 0.058841
[LightGBM] [Warning] No further splits wit

 28%|██▊       | 9/32 [00:00<00:00, 30.18it/s]

CategoricalNB model failed to execute
Negative values in data passed to CategoricalNB (input X).
FixedThresholdClassifier model failed to execute
FixedThresholdClassifier.__init__() missing 1 required positional argument: 'estimator'


100%|██████████| 32/32 [00:00<00:00, 40.88it/s]

SelfTrainingClassifier model failed to execute
You must pass an estimator to SelfTrainingClassifier. Use `estimator`.
StackingClassifier model failed to execute
StackingClassifier.__init__() missing 1 required positional argument: 'estimators'
TunedThresholdClassifierCV model failed to execute
TunedThresholdClassifierCV.__init__() missing 1 required positional argument: 'estimator'
XGBClassifier model failed to execute
'super' object has no attribute '__sklearn_tags__'
[LightGBM] [Info] Number of positive: 98, number of negative: 102
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000036 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 644
[LightGBM] [Info] Number of data points in the train set: 200, number of used features: 11
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.490000 -> initscore=-0.040005
[LightGBM] [Info] Start training from score -0.040005
[LightGBM] [Warning] No further splits w

In [14]:
for i in test.keys():
    print(f'Modelos del género {i}')
    display(test[i])  # Esto imprimirá los resultados de LazyClassifier

Modelos del género rock


,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
NearestCentroid,0.60,0.60,0.60,0.60,0.02
LinearSVC,0.59,0.59,0.59,0.59,0.02
RidgeClassifier,0.59,0.59,0.59,0.59,0.01
LinearDiscriminantAnalysis,0.59,0.59,0.59,0.59,0.04
RidgeClassifierCV,0.59,0.59,0.59,0.59,0.01
CalibratedClassifierCV,0.59,0.59,0.59,0.59,0.06
LogisticRegression,0.58,0.58,0.58,0.58,0.02
AdaBoostClassifier,0.58,0.58,0.58,0.58,0.11
GaussianNB,0.56,0.57,0.57,0.53,0.01


Modelos del género pop


,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
LGBMClassifier,0.61,0.61,0.61,0.61,0.05
SVC,0.61,0.61,0.61,0.61,0.02
DecisionTreeClassifier,0.61,0.61,0.61,0.61,0.02
QuadraticDiscriminantAnalysis,0.60,0.61,0.61,0.59,0.01
NuSVC,0.60,0.60,0.60,0.60,0.04
RidgeClassifier,0.60,0.60,0.60,0.60,0.01
LinearSVC,0.60,0.60,0.60,0.60,0.02
LinearDiscriminantAnalysis,0.60,0.60,0.60,0.60,0.03
LogisticRegression,0.59,0.59,0.59,0.59,0.03


Modelos del género rap


...

,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
NuSVC,0.57,0.58,0.58,0.57,0.02
LGBMClassifier,0.56,0.57,0.57,0.55,0.03
QuadraticDiscriminantAnalysis,0.57,0.56,0.56,0.57,0.01
ExtraTreesClassifier,0.54,0.55,0.55,0.54,0.08
KNeighborsClassifier,0.54,0.54,0.54,0.54,0.01
BernoulliNB,0.52,0.53,0.53,0.52,0.01
DecisionTreeClassifier,0.52,0.53,0.53,0.52,0.01
AdaBoostClassifier,0.50,0.51,0.51,0.50,0.10
SVC,0.50,0.50,0.50,0.50,0.01


Modelos del género reggaeton


,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
AdaBoostClassifier,0.63,0.64,0.64,0.64,0.09
DecisionTreeClassifier,0.60,0.58,0.58,0.59,0.02
LGBMClassifier,0.58,0.57,0.57,0.58,0.03
SVC,0.56,0.57,0.57,0.56,0.01
PassiveAggressiveClassifier,0.58,0.56,0.56,0.57,0.02
NearestCentroid,0.56,0.56,0.56,0.56,0.02
RandomForestClassifier,0.56,0.55,0.55,0.56,0.12
BaggingClassifier,0.56,0.55,0.55,0.56,0.03
QuadraticDiscriminantAnalysis,0.50,0.54,0.54,0.43,0.01


Modelos del género r&b


,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
RandomForestClassifier,0.62,0.62,0.62,0.62,0.12
AdaBoostClassifier,0.60,0.60,0.60,0.60,0.09
QuadraticDiscriminantAnalysis,0.60,0.59,0.59,0.59,0.01
Perceptron,0.58,0.58,0.58,0.58,0.01
LogisticRegression,0.54,0.55,0.55,0.54,0.02
RidgeClassifierCV,0.54,0.55,0.55,0.54,0.01
ExtraTreesClassifier,0.54,0.54,0.54,0.54,0.09
SVC,0.52,0.52,0.52,0.52,0.01
LabelPropagation,0.50,0.52,0.52,0.48,0.02


## Genero Filmi

In [15]:
genero='filmi'
df_temp = df_final.copy()
df_temp[genero] = df_temp['Genero'].apply(lambda x: busca_genero(genero, x))
df_gen = df_temp[df_temp[genero] == 1].copy()
    
    
    

    # Calcular el tercer cuartil (Q3)
q3 = df_gen['Stream'].quantile(0.75)

    # Definir las categorías
bins = [0, q3, df_gen['Stream'].max()]
labels = [0, 1]

df_gen['Categoria'] = pd.cut(df_gen['Stream'], bins=bins, labels=labels, include_lowest=True)

    # Verificar si hay suficientes muestras en la categoría 1
count_cat1 = df_gen['Categoria'].value_counts().get(1, 0)

  

    # Filtrar por categorías
categoria_0 = df_gen[df_gen['Categoria'] == 0]
categoria_1 = df_gen[df_gen['Categoria'] == 1]

    # Muestrear la categoría 0 con el mismo número de elementos que la categoría 1
categoria_0_sample = categoria_0.sample(n=min(count_cat1, len(categoria_0)), random_state=42)

    # Concatenar para obtener un dataset balanceado
df_balanceado = pd.concat([categoria_0_sample, categoria_1]).sample(frac=1, random_state=42).reset_index(drop=True)

    # Definir variables predictoras y objetivo
X = df_balanceado[['Acousticness', 'Danceability', 'Duration_min', 'Energy',
                       'Instrumentalness', 'Key', 'Liveness', 'Loudness',
                       'Speechiness', 'Tempo', 'Valence']]
y = df_balanceado['Categoria']

    # Dividir en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [20]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd



# Definir el modelo
clf = ExtraTreesClassifier(random_state=42)

# Definir el grid de hiperparámetros a probar
param_grid = {
    'n_estimators': [50, 100, 200],  # Número de árboles
    'max_depth': [None, 10, 20, 30],  # Profundidad máxima
    'min_samples_split': [2, 5, 10],  # Mínimo de muestras para dividir
    'min_samples_leaf': [1, 2, 4],  # Mínimo de muestras por hoja
    'criterion': ['gini', 'entropy'],  # Función de medición
}

# Configurar GridSearchCV
grid_search = GridSearchCV(estimator=clf, param_grid=param_grid, 
                           cv=5, n_jobs=-1, verbose=1, scoring='accuracy')

# Ajustar el modelo a los datos
grid_search.fit(X_train, y_train)

# Mejor combinación de hiperparámetros
print("Mejores parámetros encontrados:", grid_search.best_params_)

# Evaluar en el conjunto de prueba
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Precisión en el conjunto de prueba:", accuracy)
# Obtener los mejores hiperparámetros y evaluar en el conjunto de prueba
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

# Mostrar los mejores hiperparámetros encontrados
print("Mejores hiperparámetros:", grid_search.best_params_)

# Evaluar el modelo con las métricas de clasificación
print("\nInforme de clasificación:")
print(classification_report(y_test, y_pred))

Fitting 5 folds for each of 216 candidates, totalling 1080 fits
Mejores parámetros encontrados: {'criterion': 'gini', 'max_depth': 10, 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 50}
Precisión en el conjunto de prueba: 0.5652173913043478
Mejores hiperparámetros: {'criterion': 'gini', 'max_depth': 10, 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 50}

Informe de clasificación:
              precision    recall  f1-score   support

           0       0.50      0.53      0.52        30
           1       0.62      0.59      0.61        39

    accuracy                           0.57        69
   macro avg       0.56      0.56      0.56        69
weighted avg       0.57      0.57      0.57        69



Conclusiones del Modelo ExtraTreesClassifier
1️⃣ El modelo tiene un rendimiento moderado con una precisión del 56.5%.

La precisión general sigue estando baja, lo que sugiere que el modelo no está diferenciando bien entre clases.
La precisión de la clase 1 es de 0.62, lo que indica que el modelo clasifica bien los casos positivos, pero aún comete errores.
El recall de la clase 1 es 0.59, lo que significa que el modelo detecta un 59% de los ejemplos reales de la clase 1.

2️⃣ El modelo detecta mejor la clase 1 que la clase 0.

La clase 1 tiene un mejor f1-score (0.61) en comparación con la clase 0 (0.52).
La clase 0 tiene una precisión de 0.50 y un recall de 0.53, lo que indica que el modelo tiene problemas para identificar correctamente esta clase.
Esto sugiere que el modelo está sesgado hacia la clase 1, pero sigue sin ser suficientemente preciso.

## Rock
Vamos a usar el NearestCentroid

In [26]:
genero='rock'
df_temp = df_final.copy()
df_temp[genero] = df_temp['Genero'].apply(lambda x: busca_genero(genero, x))
df_gen = df_temp[df_temp[genero] == 1].copy()
    
    
    

    # Calcular el tercer cuartil (Q3)
q3 = df_gen['Stream'].quantile(0.75)

    # Definir las categorías
bins = [0, q3, df_gen['Stream'].max()]
labels = [0, 1]

df_gen['Categoria'] = pd.cut(df_gen['Stream'], bins=bins, labels=labels, include_lowest=True)

    # Verificar si hay suficientes muestras en la categoría 1
count_cat1 = df_gen['Categoria'].value_counts().get(1, 0)

  

    # Filtrar por categorías
categoria_0 = df_gen[df_gen['Categoria'] == 0]
categoria_1 = df_gen[df_gen['Categoria'] == 1]

    # Muestrear la categoría 0 con el mismo número de elementos que la categoría 1
categoria_0_sample = categoria_0.sample(n=min(count_cat1, len(categoria_0)), random_state=42)

    # Concatenar para obtener un dataset balanceado
df_balanceado = pd.concat([categoria_0_sample, categoria_1]).sample(frac=1, random_state=42).reset_index(drop=True)

    # Definir variables predictoras y objetivo
X = df_balanceado[['Acousticness', 'Danceability', 'Duration_min', 'Energy',
                       'Instrumentalness', 'Key', 'Liveness', 'Loudness',
                       'Speechiness', 'Tempo', 'Valence']]
y = df_balanceado['Categoria']

    # Dividir en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [29]:
from sklearn.neighbors import NearestCentroid
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import classification_report, accuracy_score
import numpy as np



# Definir el modelo NearestCentroid
clf = NearestCentroid()

# Definir el grid de hiperparámetros a probar
param_grid = {
    'metric': ['euclidean', 'manhattan', 'cosine'],  # Distancias a probar
    'shrink_threshold': [None, 0.1, 0.5, 1.0, 2.0]  # Regularización (None significa sin regularización)
}

# Configurar GridSearchCV
grid_search = GridSearchCV(
    estimator=clf,
    param_grid=param_grid,
    cv=5,
    n_jobs=-1,
    verbose=1,
    scoring='f1'  # Puedes cambiar a 'accuracy', 'recall', etc.
)

# Ajustar el modelo a los datos
grid_search.fit(X_train, y_train)

# 🔹 Mejor combinación de hiperparámetros
print("Mejores parámetros encontrados:", grid_search.best_params_)

# Evaluar en el conjunto de prueba
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

# 🔹 Reporte de métricas
print("\nInforme de clasificación:")
print(classification_report(y_test, y_pred))
print("Precisión en el conjunto de prueba:", accuracy_score(y_test, y_pred))


Fitting 5 folds for each of 15 candidates, totalling 75 fits
Mejores parámetros encontrados: {'metric': 'euclidean', 'shrink_threshold': 2.0}

Informe de clasificación:
              precision    recall  f1-score   support

           0       0.59      0.49      0.53        80
           1       0.55      0.65      0.60        77

    accuracy                           0.57       157
   macro avg       0.57      0.57      0.56       157
weighted avg       0.57      0.57      0.56       157

Precisión en el conjunto de prueba: 0.5668789808917197


In [30]:
from sklearn.neighbors import NearestCentroid
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import StandardScaler
import numpy as np

# Normalizar los datos antes de usar NearestCentroid
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Definir el modelo NearestCentroid
clf = NearestCentroid()

# Definir el grid de hiperparámetros a probar
param_grid = {
    'metric': ['euclidean', 'manhattan', 'cosine'],  
    'shrink_threshold': [0.0, 0.1, 0.5, 1.0, 2.0]  
}

# Configurar GridSearchCV optimizando el recall de la clase 1
grid_search = GridSearchCV(
    estimator=clf,
    param_grid=param_grid,
    cv=5,
    n_jobs=-1,
    verbose=1,
    scoring='recall'
)

# Ajustar el modelo a los datos
grid_search.fit(X_train, y_train)

# 🔹 Mejor combinación de hiperparámetros
print("Mejores parámetros encontrados:", grid_search.best_params_)

# Evaluar en el conjunto de prueba
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

# 🔹 Reporte de métricas
print("\nInforme de clasificación:")
print(classification_report(y_test, y_pred))
print("Precisión en el conjunto de prueba:", accuracy_score(y_test, y_pred))


Fitting 5 folds for each of 15 candidates, totalling 75 fits
Mejores parámetros encontrados: {'metric': 'euclidean', 'shrink_threshold': 0.1}

Informe de clasificación:
              precision    recall  f1-score   support

           0       0.61      0.55      0.58        80
           1       0.58      0.64      0.60        77

    accuracy                           0.59       157
   macro avg       0.59      0.59      0.59       157
weighted avg       0.59      0.59      0.59       157

Precisión en el conjunto de prueba: 0.5923566878980892


 Conclusiones del Modelo NearestCentroid

1️⃣ El modelo alcanzó una precisión global del 59.2%.

La accuracy del modelo es 0.5923, lo que indica que acierta en aproximadamente 59% de los casos.
No es un resultado excelente, pero puede ser aceptable dependiendo del problema y la distribución de datos.

2️⃣ El modelo tiene un desempeño similar en ambas clases.

Clase 0: Precisión 0.61, Recall 0.55, F1-score 0.58.
Clase 1: Precisión 0.58, Recall 0.64, F1-score 0.60.
La diferencia entre las clases no es grande, pero la clase 1 tiene mejor recall (0.64), lo que significa que el modelo detecta más casos positivos que negativos.